## 전처리

In [1]:
import pandas as pd
from ast import literal_eval

movies = pd.read_csv('./data/movies_metadata.csv')[['id', 'title', 'genres', 'vote_average']]
movies['genres'] = movies['genres'].apply(literal_eval).apply(lambda genres : ', '.join([g['name'] for g in genres]))
movies.head(5)

/var/folders/yv/z9fdg8sd36x4jfwfzp1463xh0000gq/T/ipykernel_21894/3029125766.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('./data/movies_metadata.csv')[['id', 'title', 'genres', 'vote_average']]


,id,title,genres,vote_average
0,862,Toy Story,"Animation, Comedy, Family",7.7
1,8844,Jumanji,"Adventure, Fantasy, Family",6.9
2,15602,Grumpier Old Men,"Romance, Comedy",6.5
3,31357,Waiting to Exhale,"Comedy, Drama, Romance",6.1
4,11862,Father of the Bride Part II,Comedy,5.7


## Embedding

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device('mps') if torch.cuda.is_available() else torch.device('cpu')
sbert_model_name = "jhgan/ko-sroberta-multitask"
model = SentenceTransformer(sbert_model_name).to(device=device)
embeddings = model.encode(sentences, convert_to_numpy=True)

In [ ]:
from milvus import default_server
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
from sentence_transformers import SentenceTransformer

with default_server:
    default_server.set_base_dir('milvus_data')

    # 서버 연결
    connections.connect(host='127.0.0.1', port=default_server.listen_port)

    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="title", dtype=DataType.VARCHAR),
        FieldSchema(name="genres", dtype=DataType.VARCHAR),
        FieldSchema(name="vote_average", dtype=DataType.DOUBLE),
        FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
    ]
    schema = CollectionSchema(fields, "movie")
    hello_milvus = Collection("movie", schema)

    model = SentenceTransformer('jhgan/ko-sroberta-multitask')